### **Preparation**
- install pyhealth alpha version


In [ ]:
# !pip3 install pyhealth==1.0a2
# !nvidia-smi
!git clone -b v1.0a2 https://github.com/sunlabuiuc/PyHealth.git
!pip install PyHealth
!nvidia-smi

Cloning into 'PyHealth'...
fatal: Remote branch v1.0a2 not found in upstream origin
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 1.6 MB/s 
     |████████████████████████████████| 29.3 MB 1.4 MB/s 
Fri Dec  2 20:13:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    30W /  70W

### **Step 1: Load dataset**
- **[README]:** We call [pyhealth.datasets](http://34.172.253.102/api/datasets.html) to process and obtain the dataset. 
  - `root` is the arguments directing to the data folder. 
  - `tables` is a list of table names from raw databases, which specifies the information that will be used in building the pipeline. Currently, we provide [MIMIC3Dataset](http://34.172.253.102/api/datasets/pyhealth.datasets.MIMIC3Dataset.html), [MIMIC4Dataset](http://34.172.253.102/api/datasets/pyhealth.datasets.MIMIC4Dataset.html), [eICUDataset](http://34.172.253.102/api/datasets/pyhealth.datasets.eICUDataset.html), [OMOPDataset](http://34.172.253.102/api/datasets/pyhealth.datasets.OMOPDataset.html).
  - `code_mapping [default: None]` asks a directionary input, specifying the new coding systems for each data table. For example, `{"NDC": "ATC4"}` means that our pyhealth will automatically change the codings from `NDC` into ATC-4 level for tables if any.
- **[Next Step]:** This `pyhealth.datasets` object will be used in **Step 2**.
- **[Advanced Use Case]:** Researchers can use the dict-based output alone `dataset.patients` alone for supporting their own tasks.

In [ ]:
from pyhealth.datasets import MIMIC3Dataset

mimic3_ds = MIMIC3Dataset(
        root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
        tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
        code_mapping={"NDC": "ATC"},
        refresh_cache=False,
)

mimic3_ds.stat()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
NumExpr defaulting to 2 threads.
Mapping codes: 100%|██████████| 49993/49993 [00:39<00:00, 1262.03it/s]


Statistics of MIMIC3Dataset dataset (dev=False):
	- Number of patients: 49993
	- Number of visits: 52769
	- Number of visits per patient: 1.0555
	- Number of events per visit in DIAGNOSES_ICD: 9.1038
	- Number of events per visit in PROCEDURES_ICD: 3.2186
	- Number of events per visit in PRESCRIPTIONS: 32.9969


'Statistics of MIMIC3Dataset dataset (dev=False):\n\t- Number of patients: 49993\n\t- Number of visits: 52769\n\t- Number of visits per patient: 1.0555\n\t- Number of events per visit in DIAGNOSES_ICD: 9.1038\n\t- Number of events per visit in PROCEDURES_ICD: 3.2186\n\t- Number of events per visit in PRESCRIPTIONS: 32.9969'

In [ ]:
# data format
mimic3_ds.info()


dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info



### **Step 2: Define ML task**
- reference: [pyhealth.tasks](http://34.172.253.102/api/tasks.html)
- split data

In [ ]:
from pyhealth.tasks import mortality_prediction_mimic3_fn

mimic3_ds.set_task(task_fn=mortality_prediction_mimic3_fn)
# stats info
mimic3_ds.stat()

Generating samples for mortality_prediction_mimic3_fn: 100%|██████████| 49993/49993 [00:00<00:00, 390602.99it/s]


Statistics of mortality_prediction_mimic3_fn task:
	- Dataset: MIMIC3Dataset (dev=False)
	- Number of samples: 2189
	- Number of patients: 2112
	- Number of visits: 2189
	- Number of visits per patient: 1.0365
	- conditions:
		- Number of conditions per sample: 10.3444
		- Number of unique conditions: 2524
		- Distribution of conditions (Top-10): [('4019', 875), ('41401', 591), ('42731', 536), ('25000', 482), ('4280', 471), ('2724', 324), ('5849', 297), ('53081', 280), ('5990', 262), ('2720', 254)]
	- procedures:
		- Number of procedures per sample: 4.0822
		- Number of unique procedures: 813
		- Distribution of procedures (Top-10): [('3893', 458), ('3961', 419), ('9904', 393), ('8856', 250), ('3615', 234), ('9604', 222), ('966', 213), ('8872', 208), ('9671', 180), ('3722', 177)]
	- drugs:
		- Number of drugs per sample: 34.9603
		- Number of unique drugs: 857
		- Distribution of drugs (Top-10): [('B05XA03', 1610), ('V06DC01', 1381), ('N02BE01', 1329), ('A06AA02', 1309), ('B01AB01', 13

"Statistics of mortality_prediction_mimic3_fn task:\n\t- Dataset: MIMIC3Dataset (dev=False)\n\t- Number of samples: 2189\n\t- Number of patients: 2112\n\t- Number of visits: 2189\n\t- Number of visits per patient: 1.0365\n\t- conditions:\n\t\t- Number of conditions per sample: 10.3444\n\t\t- Number of unique conditions: 2524\n\t\t- Distribution of conditions (Top-10): [('4019', 875), ('41401', 591), ('42731', 536), ('25000', 482), ('4280', 471), ('2724', 324), ('5849', 297), ('53081', 280), ('5990', 262), ('2720', 254)]\n\t- procedures:\n\t\t- Number of procedures per sample: 4.0822\n\t\t- Number of unique procedures: 813\n\t\t- Distribution of procedures (Top-10): [('3893', 458), ('3961', 419), ('9904', 393), ('8856', 250), ('3615', 234), ('9604', 222), ('966', 213), ('8872', 208), ('9671', 180), ('3722', 177)]\n\t- drugs:\n\t\t- Number of drugs per sample: 34.9603\n\t\t- Number of unique drugs: 857\n\t\t- Distribution of drugs (Top-10): [('B05XA03', 1610), ('V06DC01', 1381), ('N02BE0

In [ ]:
from pyhealth.datasets.splitter import split_by_patient
from torch.utils.data import DataLoader
from pyhealth.datasets.utils import collate_fn_dict

# data split
train_dataset, val_dataset, test_dataset = split_by_patient(mimic3_ds, [0.8, 0.1, 0.1])

# create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn_dict)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn_dict)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn_dict)

### **Step 3: Define ML Model**
- RETAIN
- RNN
- CNN

##### **RETAIN**


In [ ]:
import torch
print(torch.cuda.device_count())
device = torch.device('cuda:0')

1


In [ ]:
from pyhealth.models import RETAIN

# device = "cuda:0"

model = RETAIN(
    dataset=mimic3_ds,
    # feature_keys=["conditions", "procedures", "drugs"],
    feature_keys=["conditions"],
    label_key="label",
    operation_level='visit',
    mode="binary",
)
model.to(device)


RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
pwd '"../'

'/content'

In [ ]:
!mkdir "../output/"

mkdir: cannot create directory ‘../output/’: File exists


In [ ]:
!ls "../output/"

In [ ]:
ls "../output/20221201-175007"

last.ckpt  log.txt


In [ ]:
from pyhealth.trainer import Trainer
# from pyhealth.metrics import average_precision_score
import torch

# STEP 4: define trainer
trainer = Trainer(model=model, enable_logging=True, output_path="../output", device=device)
trainer.train(
    # model,
    train_dataloader=train_loader,
    epochs=20,
    optimizer_class=torch.optim.Adam,
    optimizer_params={"lr": 1e-3, "weight_decay": 1e-5},
    val_dataloader=val_loader,
    # val_metric=average_precision_score,
    # monitor='True'
)

INFO:root:RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)
RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_feature

Epoch 0 / 30:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:root:--- Train epoch-0, step-28 ---
--- Train epoch-0, step-28 ---
INFO:root:loss: 0.4054
loss: 0.4054


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 166.35it/s]
INFO:root:--- Eval epoch-0, step-28 ---
--- Eval epoch-0, step-28 ---
INFO:root:pr_auc: 0.0476
pr_auc: 0.0476
INFO:root:roc_auc: 0.5100
roc_auc: 0.5100
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.2380
loss: 0.2380


Epoch 1 / 30:   0%|          | 0/28 [00:00<?, ?it/s]

...

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 166.37it/s]
INFO:root:--- Eval epoch-28, step-812 ---
--- Eval epoch-28, step-812 ---
INFO:root:pr_auc: 0.0342
pr_auc: 0.0342
INFO:root:roc_auc: 0.4292
roc_auc: 0.4292
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.2883
loss: 0.2883


Epoch 29 / 30:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:root:--- Train epoch-29, step-840 ---
--- Train epoch-29, step-840 ---
INFO:root:loss: 0.0077
loss: 0.0077


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 150.85it/s]
INFO:root:--- Eval epoch-29, step-840 ---
--- Eval epoch-29, step-840 ---
INFO:root:pr_auc: 0.0353
pr_auc: 0.0353
INFO:root:roc_auc: 0.4458
roc_auc: 0.4458
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.2857
loss: 0.2857
INFO:root:Loaded best model
Loaded best model


FileNotFoundError: ignored

In [ ]:
y_true_all, y_prob_all, loss_mean = trainer.inference(test_loader)
print('loss_mean', loss_mean)
from pyhealth.metrics import binary_metrics_fn
# metrics = ["pr_auc", "roc_auc", "f1", "accuracy"]
metrics = ["pr_auc", "roc_auc",  "accuracy"]
binary_metrics_fn(y_true_all, y_prob_all, metrics=metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 45.74it/s]

loss_mean 0.15017623617313802


{'pr_auc': 0.017304240496413284,
 'roc_auc': 0.34375,
 'accuracy': 0.9818181818181818}

##### **RNN**


In [ ]:
from pyhealth.models import RNN

# device = "cuda:0"

model = RNN(
    dataset=mimic3_ds,
    feature_keys=["conditions"],
    label_key="label",
    operation_level='visit',
    mode="binary",
)
model.to(device)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True, dropout=0.5)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
from pyhealth.trainer import Trainer
# from pyhealth.metrics import average_precision_score
import torch

# STEP 4: define trainer
trainer = Trainer(model=model, enable_logging=True, output_path="../output", device=device)
trainer.train(
    # model,
    train_dataloader=train_loader,
    epochs=40,
    optimizer_class=torch.optim.Adam,
    optimizer_params={"lr": 1e-3, "weight_decay": 1e-5},
    val_dataloader=val_loader,
    # val_metric=average_precision_score,
    # monitor='True'
)

INFO:root:RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True, dropout=0.5)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)
RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True, dropout=0.5)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)
INFO:root:Metrics: None
Metrics: None
INFO:root:Device: cuda:0
Device: cuda:0
INFO:root:

INFO:root:Training:
Training:
INFO:root:Batch size: 64
Batch size: 64
INFO:root:Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer: <class 'torch.optim.adam.Adam'>
INFO:root:Optimizer params: {'lr': 0.001, 'weight_decay': 1e-05}
Optimizer params: {'lr': 

Epoch 0 / 40:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:root:--- Train epoch-0, step-28 ---
--- Train epoch-0, step-28 ---
INFO:root:loss: 0.0251
loss: 0.0251


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 442.10it/s]
INFO:root:--- Eval epoch-0, step-28 ---
--- Eval epoch-0, step-28 ---
INFO:root:pr_auc: 0.0332
pr_auc: 0.0332
INFO:root:roc_auc: 0.3821
roc_auc: 0.3821
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.3275
loss: 0.3275


Epoch 1 / 40:   0%|          | 0/28 [00:00<?, ?it/s]

...

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 286.21it/s]
INFO:root:--- Eval epoch-38, step-1092 ---
--- Eval epoch-38, step-1092 ---
INFO:root:pr_auc: 0.0306
pr_auc: 0.0306
INFO:root:roc_auc: 0.3632
roc_auc: 0.3632
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.5093
loss: 0.5093


Epoch 39 / 40:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:root:--- Train epoch-39, step-1120 ---
--- Train epoch-39, step-1120 ---
INFO:root:loss: 0.0024
loss: 0.0024


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 282.15it/s]
INFO:root:--- Eval epoch-39, step-1120 ---
--- Eval epoch-39, step-1120 ---
INFO:root:pr_auc: 0.0307
pr_auc: 0.0307
INFO:root:roc_auc: 0.3614
roc_auc: 0.3614
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.5146
loss: 0.5146
INFO:root:Loaded best model
Loaded best model


FileNotFoundError: ignored

In [ ]:
y_true_all, y_prob_all, loss_mean = trainer.inference(test_loader)
from pyhealth.metrics import binary_metrics_fn
# metrics = ["pr_auc", "roc_auc", "f1", "accuracy"]
print('loss_mean', loss_mean)
metrics = ["pr_auc", "roc_auc",  "accuracy"]
binary_metrics_fn(y_true_all, y_prob_all, metrics=metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 254.54it/s]

loss_mean 0.24299793658519775


{'pr_auc': 0.01567816899973511,
 'roc_auc': 0.2974537037037037,
 'accuracy': 0.9818181818181818}

##### **CNN**


In [ ]:
from pyhealth.models import CNN

model = CNN(
    dataset=mimic3_ds,
    feature_keys=["conditions"],
    label_key="label",
    operation_level='visit',
    mode="binary",
)
model.to(device)

CNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (cnn): ModuleDict(
    (conditions): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool1d(output_size=1)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
from pyhealth.trainer import Trainer
# from pyhealth.metrics import average_precision_score
import torch

# STEP 4: define trainer
trainer = Trainer(model=model, enable_logging=True, output_path="../output", device=device)
trainer.train(
    # model,
    train_dataloader=train_loader,
    epochs=30,
    optimizer_class=torch.optim.Adam,
    optimizer_params={"lr": 1e-3, "weight_decay": 1e-5},
    val_dataloader=val_loader,
    # val_metric=average_precision_score,
    # monitor='True'
)

INFO:root:CNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (cnn): ModuleDict(
    (conditions): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool1d(output_size=1)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)
CNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (cnn): ModuleDict(
    (conditions): CNNLayer(
      (cnn): ModuleDict(
     

Epoch 0 / 30:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:root:--- Train epoch-0, step-28 ---
--- Train epoch-0, step-28 ---
INFO:root:loss: 0.2482
loss: 0.2482


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 266.66it/s]
INFO:root:--- Eval epoch-0, step-28 ---
--- Eval epoch-0, step-28 ---
INFO:root:pr_auc: 0.1665
pr_auc: 0.1665
INFO:root:roc_auc: 0.5118
roc_auc: 0.5118
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.1625
loss: 0.1625


Epoch 1 / 30:   0%|          | 0/28 [00:00<?, ?it/s]

...

INFO:root:loss: 0.0006
loss: 0.0006
Evaluation: 100%|██████████| 4/4 [00:00<00:00, 271.30it/s]
INFO:root:--- Eval epoch-28, step-812 ---
--- Eval epoch-28, step-812 ---
INFO:root:pr_auc: 0.0544
pr_auc: 0.0544
INFO:root:roc_auc: 0.5867
roc_auc: 0.5867
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.2557
loss: 0.2557


Epoch 29 / 30:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:root:--- Train epoch-29, step-840 ---


--- Train epoch-29, step-840 ---
INFO:root:loss: 0.0005
loss: 0.0005
Evaluation: 100%|██████████| 4/4 [00:00<00:00, 237.54it/s]
INFO:root:--- Eval epoch-29, step-840 ---
--- Eval epoch-29, step-840 ---
INFO:root:pr_auc: 0.0570
pr_auc: 0.0570
INFO:root:roc_auc: 0.6032
roc_auc: 0.6032
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 0.2570
loss: 0.2570
INFO:root:Loaded best model
Loaded best model


FileNotFoundError: ignored

In [ ]:
y_true_all, y_prob_all, loss_mean = trainer.inference(test_loader)
print('loss_mean', loss_mean)
from pyhealth.metrics import binary_metrics_fn
# metrics = ["pr_auc", "roc_auc", "f1", "accuracy"]
metrics = ["pr_auc", "roc_auc",  "accuracy"]
binary_metrics_fn(y_true_all, y_prob_all, metrics=metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 256.83it/s]

loss_mean 0.13101924117654562


{'pr_auc': 0.048972010378174384,
 'roc_auc': 0.4212962962962963,
 'accuracy': 0.9818181818181818}

##### **Transformer**


In [ ]:

from pyhealth.models import Transformer

model = Transformer(
    dataset=mimic3_ds,
    feature_keys=["conditions"],
    label_key="label",
    operation_level='visit',
    mode="binary",
)
model.to(device)

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0): Linear(in_features=128, out_features=128, bias=False)
              (1): Linear(in_features=128, out_features=128, bias=False)
              (2): Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activat

In [ ]:
from pyhealth.trainer import Trainer
# from pyhealth.metrics import average_precision_score
import torch

# STEP 4: define trainer
trainer = Trainer(model=model, enable_logging=True, output_path="../output", device=device)
trainer.train(
    # model,
    train_dataloader=train_loader,
    epochs=40,
    optimizer_class=torch.optim.Adam,
    optimizer_params={"lr": 1e-3, "weight_decay": 1e-5},
    val_dataloader=val_loader,
    # val_metric=average_precision_score,
    # monitor='True'
)

INFO:root:Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2526, 128, padding_idx=0)
  )
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0): Linear(in_features=128, out_features=128, bias=False)
              (1): Linear(in_features=128, out_features=128, bias=False)
              (2): Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
          

Epoch 0 / 40:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:root:--- Train epoch-0, step-28 ---
--- Train epoch-0, step-28 ---
INFO:root:loss: 0.0017
loss: 0.0017


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 120.21it/s]
INFO:root:--- Eval epoch-0, step-28 ---
--- Eval epoch-0, step-28 ---
INFO:root:pr_auc: 0.0332
pr_auc: 0.0332
INFO:root:roc_auc: 0.4051
roc_auc: 0.4051
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 1.0220
loss: 1.0220


Epoch 1 / 40:   0%|          | 0/28 [00:00<?, ?it/s]

...

INFO:root:--- Train epoch-38, step-1092 ---


--- Train epoch-38, step-1092 ---
INFO:root:loss: 0.0001
loss: 0.0001
Evaluation: 100%|██████████| 4/4 [00:00<00:00, 303.13it/s]
INFO:root:--- Eval epoch-38, step-1092 ---
--- Eval epoch-38, step-1092 ---
INFO:root:pr_auc: 0.0322
pr_auc: 0.0322
INFO:root:roc_auc: 0.3992
roc_auc: 0.3992
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 2.5464
loss: 2.5464


Epoch 39 / 40:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:root:--- Train epoch-39, step-1120 ---
--- Train epoch-39, step-1120 ---
INFO:root:loss: 0.0000
loss: 0.0000
Evaluation: 100%|██████████| 4/4 [00:00<00:00, 271.03it/s]
INFO:root:--- Eval epoch-39, step-1120 ---
--- Eval epoch-39, step-1120 ---
INFO:root:pr_auc: 0.0322
pr_auc: 0.0322
INFO:root:roc_auc: 0.3992
roc_auc: 0.3992
INFO:root:f1: 0.0000
f1: 0.0000
INFO:root:loss: 2.5501
loss: 2.5501
INFO:root:Loaded best model
Loaded best model


FileNotFoundError: ignored

In [ ]:
y_true_all, y_prob_all, loss_mean = trainer.inference(test_loader)
from pyhealth.metrics import binary_metrics_fn
# metrics = ["pr_auc", "roc_auc", "f1", "accuracy"]
metrics = ["pr_auc", "roc_auc",  "accuracy"]
binary_metrics_fn(y_true_all, y_prob_all, metrics=metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 223.10it/s]


{'pr_auc': 0.035853756795071856,
 'roc_auc': 0.6840277777777778,
 'accuracy': 0.9772727272727273}

In [ ]:
loss_mean 

0.7079442888498306